In [5]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()

In [115]:
class Lotteria:
    def __init__(self,
                 max_num=90,
                 max_sup=90,
                 length=6,
                 superstar=0):
        self.max_numeri = max_num
        self.max_superstar = max_sup
        self.length = length
        self.superstar = superstar
        self.numeri = list(range(1, self.max_numeri+1))
        

    @staticmethod
    def pallottoliere(maximo):
        while True:
            yield rnd.randint(1, maximo)


    def choice(self, length, *args):
        numeri = self.numeri.copy()
        estratti = []
        for _ in range(length):
            rnd.shuffle(numeri)
            estratto = rnd.choice(numeri)
            estratti.append(estratto)
            numeri.remove(estratto)

        return sorted(estratti)
    

    def sample(self, length, *args):
        return sorted(rnd.sample(self.numeri, k=length))
    

    def int(self, length, num):
        estratti = set()
        while len(estratti) < length:
            estratti.add(next(self.pallottoliere(num)))

        return sorted(estratti)
    

    def many(self, extractor, length, num):
        samples = [extractor(length, num) for s in self.numeri]
        rnd.shuffle(samples)
        estratti = sorted(rnd.choice(samples))

        return estratti
    

    def extraction(self, backend='choice', many=True):
        if backend == 'choice':
            self.backend = 'choice'
            extractor = self.choice
        if backend == 'sample':
            self.backend ='sample'
            extractor = self.sample
        if backend == 'int':
            self.backend = 'int'
            extractor = self.int

        if not many:
            estratti = extractor(self.length, self.max_numeri)

            if self.superstar:
                super = extractor(self.superstar, self.max_superstar)

        else:
            estratti = self.many(extractor, self.length, self.max_numeri)

            if self.superstar:
                super = self.many(extractor, self.superstar,
                                  self.max_superstar)

        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print(f'Estrazione del: {now}', '\nNumeri estratti:', *estratti)
        if self.superstar:
            print('Superstar:', *sorted(super))

In [116]:
superenalotto = Lotteria(90, 90, 6, 2)

In [106]:
superenalotto.extraction(backend='int', many=False)

Estrazione del: 27/05/2023 15:14 
Numeri estratti: 2 36 42 44 81 86
Superstar: 53 75


In [111]:
superenalotto.extraction()

Estrazione del: 27/05/2023 15:16 
Numeri estratti: 16 21 52 59 68 83
Superstar: 86 88


In [117]:
for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    superenalotto.extraction(backend=b, many=False)

print( '\n################# many #########################\n')

for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    superenalotto.extraction(backend=b, many=True)

### choice ###
Estrazione del: 27/05/2023 15:19 
Numeri estratti: 24 32 37 38 82 90
Superstar: 18 66
### int ###
Estrazione del: 27/05/2023 15:19 
Numeri estratti: 2 28 32 36 58 79
Superstar: 51 65
### sample ###
Estrazione del: 27/05/2023 15:19 
Numeri estratti: 11 24 26 34 37 65
Superstar: 34 68

################# many #########################

### choice ###
Estrazione del: 27/05/2023 15:19 
Numeri estratti: 3 36 38 56 65 90
Superstar: 10 65
### int ###
Estrazione del: 27/05/2023 15:19 
Numeri estratti: 4 16 22 32 47 80
Superstar: 15 38
### sample ###
Estrazione del: 27/05/2023 15:19 
Numeri estratti: 6 19 31 41 73 74
Superstar: 6 21


In [118]:
superenalotto.backend

'sample'